# Training and Testing an MDRNN on Different Models

This script tests the MDRNN on different trained models and generates some output plots.

In [ ]:
%matplotlib inline
import random
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
from context import *
import empi_mdrnn

def generate_test_performance(weights_file, net, num_steps=1500):
    """Generate num_steps of unconstrained data from a network (net), with weights_file's weights"""
    net.model.load_weights(weights_file)
    net.prepare_model_for_running()
    sample = empi_mdrnn.random_sample()
    p = net.generate_performance(sample, num_steps)
    perf_df = pd.DataFrame({'t':p.T[0], 'x':p.T[1]})
    perf_df['time'] = perf_df.t.cumsum()
    return perf_df
    
def plot_nn_output(output_df, name="unknown", colour='b'):
    """Plot the data in output_df assuming it has columns x and time"""
    sns.set_style('whitegrid')
    sns.set_context('paper')
    f, ax = plt.subplots(figsize=(8, 2))
    sns.lineplot(x='time', y='x', data=perf_df, ax=ax, legend=False, color=colour)
    ax.set_xlabel("seconds")
    ax.set_ylabel("position")
    ax.set(ylim=(-0.1, 1.1))
    sns.despine(f, left=True, bottom=True)
    #ax = perf_df.plot(x='time', y='x', kind="line", color=colour, figsize=(8,2), legend=False)
    ax.get_figure().savefig('../images/model_output/'+name+'_model_output.pdf', dpi=300, bbox_inches="tight")
    ax.get_figure().savefig('../images/model_output/'+name+'_model_output.png', dpi=300, bbox_inches="tight")

### Setup network and test

Setup network and test with each trained model. Main parameters to change here are `mdrnn_units` and `num_steps`

In [ ]:
# Model Hyperparameters
mdrnn_units = 32
mdrnn_mixes = 5
mdrnn_layers = 2
# Sampling parameters
pitemp = 1.1
sigmatemp = 0.1
num_steps = 500

empi_mdrnn.MODEL_DIR = "./models/"
net = empi_mdrnn.PredictiveMusicMDRNN(mode=empi_mdrnn.NET_MODE_RUN,
                                              dimension=2,
                                              n_hidden_units=mdrnn_units,
                                              n_mixtures=mdrnn_mixes,
                                              layers=mdrnn_layers)
net.pi_temp = pitemp
net.sigma_temp = sigmatemp

In [ ]:
# test noise model.
colour = sns.color_palette('deep')[2]
model_file = "../models/musicMDRNN-dim2-layers2-units"+str(mdrnn_units)+"-mixtures5-scale10-noise.h5"
perf_df = generate_test_performance(model_file, net, num_steps=num_steps)
display(perf_df.describe())
# Generate slightly nicer plot.
plot_nn_output(perf_df, name="noise"+str(mdrnn_units), colour=colour)

In [ ]:
# Load and test synthetic model.
colour = sns.color_palette('deep')[1]
model_file = "../models/musicMDRNN-dim2-layers2-units"+str(mdrnn_units)+"-mixtures5-scale10-synth.h5"
perf_df = generate_test_performance(model_file, net, num_steps=num_steps)
display(perf_df.describe())
# Generate nice plot.
plot_nn_output(perf_df, name="synth"+str(mdrnn_units), colour=colour)

In [ ]:
# Load and test human model.
colour = sns.color_palette('deep')[3]
model_file = "../models/musicMDRNN-dim2-layers2-units"+str(mdrnn_units)+"-mixtures5-scale10-human.h5"
perf_df = generate_test_performance(model_file, net, num_steps=num_steps)
display(perf_df.describe())
# Generate nice plot.
plot_nn_output(perf_df, name="human"+str(mdrnn_units), colour=colour)

In [ ]:
# Load and test human model on more points.
colour = sns.color_palette('deep')[3]
target_seconds = 180
human_steps = target_seconds * 500 // 20
model_file = "../models/musicMDRNN-dim2-layers2-units"+str(mdrnn_units)+"-mixtures5-scale10-human.h5"
perf_df = generate_test_performance(model_file, net, num_steps=human_steps)
display(perf_df.describe())
# Generate nice plot.
plot_nn_output(perf_df, name="human"+str(mdrnn_units)+"-"+str(human_steps)+"points", colour=colour)

# Test One Model

In [ ]:
# Model Hyperparameters
mdrnn_units = 64
mdrnn_mixes = 5
mdrnn_layers = 2
# Sampling parameters
pitemp = 1.1
sigmatemp = 0.1
# Number of steps to sample
num_steps = 2000

empi_mdrnn.MODEL_DIR = "../models/20190930/"
net = empi_mdrnn.PredictiveMusicMDRNN(mode=empi_mdrnn.NET_MODE_RUN,
                                              dimension=2,
                                              n_hidden_units=mdrnn_units,
                                              n_mixtures=mdrnn_mixes,
                                              layers=mdrnn_layers)
net.pi_temp = pitemp
net.sigma_temp = sigmatemp

# Load and test synthetic model.
model_file = "../models/musicMDRNN-dim2-layers2-units64-mixtures5-scale10-human-E01-VL-1.48.hdf5"
perf_df = generate_test_performance(model_file, net, num_steps=num_steps)
display(perf_df.describe())
# Generate slightly nicer plot.
plot_nn_output(perf_df, name="human"+str(mdrnn_units))

In [ ]:

# Model Hyperparameters
mdrnn_units = 64
mdrnn_mixes = 5
mdrnn_layers = 2
# Sampling parameters
pitemp = 1.0
sigmatemp = 0.1
# Number of steps to sample
num_steps = 1000

empi_mdrnn.MODEL_DIR = "../models/20190930/"
net = empi_mdrnn.PredictiveMusicMDRNN(mode=empi_mdrnn.NET_MODE_RUN,
                                              dimension=2,
                                              n_hidden_units=mdrnn_units,
                                              n_mixtures=mdrnn_mixes,
                                              layers=mdrnn_layers)
net.pi_temp = pitemp
net.sigma_temp = sigmatemp

# Load and test synthetic model.
model_file = "../models/musicMDRNN-dim2-layers2-units64-mixtures5-scale10-human-E71-VL-2.27.hdf5"
perf_df = generate_test_performance(model_file, net, num_steps=num_steps)
display(perf_df.describe())
# Generate slightly nicer plot.
plot_nn_output(perf_df, name="human"+str(mdrnn_units))

In [ ]:
# Model Hyperparameters
mdrnn_units = 32
mdrnn_mixes = 5
mdrnn_layers = 2
# Sampling parameters
pitemp = 1
sigmatemp = 0.01
# Number of steps to sample
num_steps = 500

empi_mdrnn.MODEL_DIR = "../models/20190930/"
net = empi_mdrnn.PredictiveMusicMDRNN(mode=empi_mdrnn.NET_MODE_RUN,
                                              dimension=2,
                                              n_hidden_units=mdrnn_units,
                                              n_mixtures=mdrnn_mixes,
                                              layers=mdrnn_layers)
net.pi_temp = pitemp
net.sigma_temp = sigmatemp

# Load and test synthetic model.
model_file = "../models/20190930/musicMDRNN-dim2-layers2-units32-mixtures5-scale10-human-E84-VL-2.78.hdf5"
perf_df = generate_test_performance(model_file, net, num_steps=num_steps)
display(perf_df.describe())
# Generate slightly nicer plot.
plot_nn_output(perf_df, name="human"+str(mdrnn_units))

# Model Hyperparameters
mdrnn_units = 64
mdrnn_mixes = 5
mdrnn_layers = 2
# Sampling parameters
pitemp = 1
sigmatemp = 0.01
# Number of steps to sample
num_steps = 500

empi_mdrnn.MODEL_DIR = "../models/20190930/"
net = empi_mdrnn.PredictiveMusicMDRNN(mode=empi_mdrnn.NET_MODE_RUN,
                                              dimension=2,
                                              n_hidden_units=mdrnn_units,
                                              n_mixtures=mdrnn_mixes,
                                              layers=mdrnn_layers)
net.pi_temp = pitemp
net.sigma_temp = sigmatemp

# Load and test synthetic model.
model_file = "../models/20190930/musicMDRNN-dim2-layers2-units64-mixtures5-scale10-human-E25-VL-1.93.hdf5"
perf_df = generate_test_performance(model_file, net, num_steps=num_steps)
display(perf_df.describe())
# Generate slightly nicer plot.
plot_nn_output(perf_df, name="human"+str(mdrnn_units))

# Model Hyperparameters
mdrnn_units = 128
mdrnn_mixes = 5
mdrnn_layers = 2
# Sampling parameters
pitemp = 1
sigmatemp = 0.01
# Number of steps to sample
num_steps = 500

empi_mdrnn.MODEL_DIR = "../models/20190930/"
net = empi_mdrnn.PredictiveMusicMDRNN(mode=empi_mdrnn.NET_MODE_RUN,
                                              dimension=2,
                                              n_hidden_units=mdrnn_units,
                                              n_mixtures=mdrnn_mixes,
                                              layers=mdrnn_layers)
net.pi_temp = pitemp
net.sigma_temp = sigmatemp

# Load and test synthetic model.
model_file = "../models/20190930/musicMDRNN-dim2-layers2-units128-mixtures5-scale10-human-E36-VL-2.66.hdf5"
perf_df = generate_test_performance(model_file, net, num_steps=num_steps)
display(perf_df.describe())
# Generate slightly nicer plot.
plot_nn_output(perf_df, name="human"+str(mdrnn_units))

# Model Hyperparameters
mdrnn_units = 256
mdrnn_mixes = 5
mdrnn_layers = 2
# Sampling parameters
pitemp = 1
sigmatemp = 0.01
# Number of steps to sample
num_steps = 500

empi_mdrnn.MODEL_DIR = "../models/20190930/"
net = empi_mdrnn.PredictiveMusicMDRNN(mode=empi_mdrnn.NET_MODE_RUN,
                                              dimension=2,
                                              n_hidden_units=mdrnn_units,
                                              n_mixtures=mdrnn_mixes,
                                              layers=mdrnn_layers)
net.pi_temp = pitemp
net.sigma_temp = sigmatemp

# Load and test synthetic model.
model_file = "../models/20190930/musicMDRNN-dim2-layers2-units256-mixtures5-scale10-human-E06-VL-1.78.hdf5"
perf_df = generate_test_performance(model_file, net, num_steps=num_steps)
display(perf_df.describe())
# Generate slightly nicer plot.
plot_nn_output(perf_df, name="human"+str(mdrnn_units))